# モジュール化したプロトタイプで動作確認

いままでのプロトタイプ調査（中間調査含む）で使用した関数群を、モジュール「LDAPrototype.py」にまとめました。

このモジュールを使用し、前回レポート「LDA_in_scikit_learn_03」と同様のシナリオで検証をしてみました。

結果としては概ね同様の挙動が確認されております。

## テストデータ作成

In [1]:
from LDAPrototype import LDATestDataGenerator

testdata_generator = LDATestDataGenerator()
testdata_generator.generate_testdata() # テストデータをサンプルCSVから読み込み
testdata_generator.generate_tf_vector_and_feature() # 読み込んだテストデータから、TFベクターと、特徴語リストを作成

Parsed questions (count=99):


### テストデータを、初期学習用と、テスト用に分割

初期学習用には、#0 〜 #95 の９６件をセレクトします。

In [2]:
tf_init = testdata_generator.tf_vector[0:96]
tf_init

<96x304 sparse matrix of type '<class 'numpy.int64'>'
	with 940 stored elements in Compressed Sparse Row format>

テストは３回行いますので、#96、#97、#98 の１件ずつセレクトします（後述）。

## 初期学習実施

分類トピック件数などのパラメーターは、後日変えられるように改修予定です。（2016/02/08）

In [3]:
from LDAPrototype import LDA

lda = LDA()
lda.init_lda_model()
answers_init = lda.initial_fit_and_transform(tf_init) 

### トピックに含まれる上位１５件の特徴語を取得

In [4]:
def print_top_words_in_topic():
    # 上位１５件の特徴語インデックス配列を取得
    # デフォルトでは、１０件のトピックに分類されています
    top_features_list = lda.get_top_features_in_topic(15) 

    # TFベクターを生成した時の特徴語リストと、特徴語インデックス配列のインデックスをマッチングすることにより変換します
    for topic_idx, top_features in enumerate(top_features_list):
        top_words_list = [testdata_generator.tf_feature_names[i] for i in top_features]
        print("Topic #%d: %s" % (topic_idx, ",".join(top_words_list)))

### 初期学習用データにより、どのようなトピックが生成されたかを確認

In [5]:
print_top_words_in_topic()

Topic #0: 表示,しまい,owa,まし,辞書,ませ,消え,ime,パスワード,あり,以前,インストール,正しく,登録,ユーザー
Topic #1: メール,にて,利用,です,フォルダ,すれ,まし,表示,いい,junk,email,ない,どう,迷惑,よう
Topic #2: 英語,存在,ダイキン,利用,いる,設定,する,教え,欲しい,にて,なり,容量,メール,付く,ログイン
Topic #3: 表示,ない,れる,画面,する,起動,アドレス,欲しい,教え,設定,ライセンス,よう,検索,メール,から
Topic #4: 表示,ほしい,誤っ,ouｔlook,件名,方法,しまっ,トレイ,操作,教え,対応,れる,受信,フィールド,削除
Topic #5: ませ,長期,メール,欲しい,owa,媒体,インストール,という,web,開き,イン,見れ,サイン,見つかり,画面
Topic #6: 届く,正常,スパムフォルダ,トレイ,よう,たい,受信,メール,にて,です,thunderbird,同様,でしょ,可能,欲しい
Topic #7: でき,にて,ない,メール,表示,アドレス,検索,送信,ませ,設定,いる,まし,する,から,thunderbird
Topic #8: ます,あり,thunderbird,機能,メール,送信,トレイ,ハードディスク,リンク,テンプレート,ドライブ,違い,として,容量,同等
Topic #9: メール,にて,教え,欲しい,方法,たい,する,thunderbird,可能,から,です,受信,送信,対応,設定


## 予測実行

「追加学習」というと分かりにくいかと思ったので「予測実行」と称することといたします。

### 予測結果表示用関数

In [6]:
def print_partial_fit_result(data_idx, answers):
    topic_idx = lda.get_hit_topic_idx(answers[0]) # トップヒットのトピックインデックスを取得
    topic_prob_rate = answers[0][topic_idx] * 100 # トップヒットのトピックに属する確率

    print("==========")
    print("Sample Text #%d ---> Topic #%d (%0.1f%%):" % (data_idx, topic_idx, topic_prob_rate))
    print(testdata_generator.origin_text[data_idx])
    print("==========")
    print_top_words_in_topic()    
    print("==========")

### 予測１回目

In [7]:
data_idx = 96 # データ #96 を使用し予測実行
tf = testdata_generator.tf_vector[data_idx] # テストデータを取り出す
answers = lda.partial_fit_and_transform(tf) # 予測実行
print_partial_fit_result(data_idx, answers) # 予測結果を表示

Sample Text #96 ---> Topic #7 (61.9%):
Ouｔlook2010にて、 アカウントを設定したが、Ouｔlookを起動すると前の方のメールパスワードを求められる。
Topic #0: 表示,しまい,owa,まし,パスワード,辞書,ませ,消え,ime,あり,以前,インストール,正しく,登録,ユーザー
Topic #1: られる,求め,メール,起動,にて,ouｔlook,パスワード,アカウント,設定,する,利用,です,フォルダ,すれ,まし
Topic #2: 求め,英語,存在,ダイキン,利用,設定,する,いる,教え,欲しい,にて,られる,なり,メール,容量
Topic #3: 表示,する,ない,起動,れる,画面,アドレス,欲しい,教え,設定,ライセンス,よう,検索,アカウント,メール
Topic #4: ouｔlook,求め,られる,パスワード,起動,アカウント,する,にて,設定,メール,表示,ほしい,誤っ,件名,方法
Topic #5: ませ,長期,メール,欲しい,owa,媒体,インストール,という,web,開き,イン,見れ,サイン,見つかり,画面
Topic #6: 届く,正常,スパムフォルダ,トレイ,よう,求め,たい,受信,メール,にて,です,られる,thunderbird,同様,でしょ
Topic #7: 設定,する,アカウント,パスワード,にて,起動,メール,求め,ouｔlook,られる,でき,ない,表示,アドレス,検索
Topic #8: ます,あり,thunderbird,機能,メール,送信,トレイ,ハードディスク,リンク,テンプレート,ドライブ,違い,として,容量,同等
Topic #9: メール,にて,教え,欲しい,方法,たい,する,thunderbird,可能,から,です,受信,送信,対応,設定


### 予測２回目

In [8]:
data_idx = 97 # データ #97 を使用し予測実行
tf = testdata_generator.tf_vector[data_idx] # テストデータを取り出す
answers = lda.partial_fit_and_transform(tf) # 予測実行
print_partial_fit_result(data_idx, answers) # 予測結果を表示

Sample Text #97 ---> Topic #3 (59.7%):
Outlook2010起動時、McAfeeのアドインメッセージが表示された、「はい」と「いいえ」の選択を求められている。対応方法を教えて欲しい。
Topic #0: 表示,しまい,owa,まし,パスワード,辞書,ませ,消え,ime,あり,以前,インストール,正しく,登録,ユーザー
Topic #1: 求め,られる,起動,メール,にて,ouｔlook,られ,パスワード,アカウント,設定,する,いいえ,mcafee,アドインメッセージ,はい
Topic #2: 求め,英語,存在,ダイキン,られ,mcafee,いいえ,いる,はい,利用,設定,する,アドインメッセージ,教え,欲しい
Topic #3: 選択,表示,いいえ,られ,アドインメッセージ,はい,起動,mcafee,欲しい,教え,求め,いる,対応,方法,する
Topic #4: ouｔlook,求め,られる,パスワード,起動,アカウント,する,にて,設定,メール,表示,ほしい,誤っ,件名,方法
Topic #5: ませ,長期,メール,欲しい,owa,媒体,インストール,という,web,開き,イン,見れ,サイン,見つかり,画面
Topic #6: 届く,いいえ,求め,正常,スパムフォルダ,トレイ,よう,mcafee,はい,たい,られ,受信,アドインメッセージ,メール,にて
Topic #7: 設定,する,アカウント,パスワード,にて,起動,メール,求め,ouｔlook,られる,でき,ない,表示,アドレス,検索
Topic #8: ます,あり,thunderbird,機能,メール,mcafee,送信,トレイ,ハードディスク,リンク,テンプレート,ドライブ,違い,として,容量
Topic #9: 方法,対応,いる,教え,欲しい,mcafee,はい,アドインメッセージ,られ,いいえ,表示,求め,選択,起動,メール


### 予測３回目

In [9]:
data_idx = 98 # データ #98 を使用し予測実行
tf = testdata_generator.tf_vector[data_idx] # テストデータを取り出す
answers = lda.partial_fit_and_transform(tf) # 予測実行
print_partial_fit_result(data_idx, answers) # 予測結果を表示

Sample Text #98 ---> Topic #2 (65.6%):
Outlook2010にて、メール本文作成時に数字を入力後、文章を入力しEnterを実行すると次の行に数字が表示され、メールが書きにくいので、解除方法を教えてほしい。
Topic #0: 表示,しまい,owa,まし,パスワード,辞書,ませ,消え,ime,あり,以前,インストール,正しく,登録,ユーザー
Topic #1: 求め,られる,起動,メール,にて,ouｔlook,られ,パスワード,アカウント,設定,する,いいえ,mcafee,アドインメッセージ,はい
Topic #2: 入力,数字,書き,本文,enter,にくい,実行,ほしい,作成,文章,解除,ので,する,表示,教え
Topic #3: 選択,表示,いいえ,られ,アドインメッセージ,はい,起動,mcafee,欲しい,教え,求め,いる,対応,方法,する
Topic #4: ouｔlook,求め,られる,パスワード,起動,アカウント,する,にて,設定,メール,表示,ほしい,誤っ,件名,方法
Topic #5: 数字,ので,解除,文章,ほしい,作成,実行,にくい,enter,入力,メール,本文,書き,表示,教え
Topic #6: 届く,いいえ,求め,正常,スパムフォルダ,トレイ,よう,mcafee,数字,はい,たい,られ,受信,文章,アドインメッセージ
Topic #7: メール,する,にて,設定,アカウント,パスワード,起動,求め,ouｔlook,られる,入力,作成,ので,解除,表示
Topic #8: ます,あり,thunderbird,機能,メール,mcafee,送信,トレイ,ハードディスク,リンク,テンプレート,ドライブ,違い,として,容量
Topic #9: 方法,教え,表示,対応,いる,欲しい,mcafee,はい,アドインメッセージ,られ,いいえ,求め,作成,ほしい,選択
